# Retail Price Optimization Analysis

Analysis and Data Quality Checks.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add parent directory to path
sys.path.append(os.path.abspath(os.path.join('..')))

from data.managament.retreiver import get_latest_data

In [ ]:
# Load data from database
df = get_latest_data()
print(f"Data Loaded. Shape: {df.shape}")
df.head()

## Summary Statistics

In [ ]:
df.describe()

### Sales Information:

* **Quantity (qty)** sold per transaction has a wide range from a minimum of 1 to a maximum of 122, averaging about 14.5. This indicates significant variability in the order sizes.
* The **total_price** of products in each transaction ranges from as low as 19.9 to a staggering 12095, suggesting a broad spectrum of order values.
* The **unit_price** averages at around 106.5, with the most expensive item costing 364 and the cheapest one being 19.9.

### Product Information:

* The **product_name_lenght** are relatively brief, ranging from 29 to 60 characters in length, averaging about 48.7 characters.
* The **product_description_lenght**, however, are much longer, averaging 767.4 characters.
* On average, a product has about 2 photos (**product_photos_qty**), which varies from 1 to 8 photos.
* The **product_weight_g** of the products also varies significantly, from as light as 100 grams to as heavy as 9750 grams.
* Customers seem reasonably satisfied with the products, as indicated by the average **product_score** of 4.08, with a maximum of 4.5.

### Customer Information:

* On average, about 81 **customers** are associated with each transaction, with the maximum number of customers in a single transaction being 339.

### Temporal Information:

* The data ranges from January 2017 to January 2018.
* Most transactions occur on **weekdays** with an average of 21.77, while **weekends** see less activity with an average of 8.66.
* **Holidays** have the least transactions with an average of 1.49.

### Competitor Information:

* The competitors' prices (**ps1, ps2, ps3**) and freight prices (**fp1, fp2, fp3**) vary, with the averages for unit prices being around 79.45, 92.93, and 84.18 respectively, and freight prices being around 4.16, 4.12, and 4 respectively. This indicates competition in pricing and shipping costs.

### Lag Price:

* The **lag_price**, which could be the price of the product in the previous period, averages around 107.4. This might be useful in understanding price trends.

### Other variables:

* **Volume**, possibly referring to the total volume of sales, averages around 10664.63.

In [ ]:
class DataQualityChecks:
    """
    A class used to perform basic data quality checks on a pandas DataFrame.
    """

    def __init__(self, df: pd.DataFrame):
        self.df = df

    def check_missing_values(self) -> pd.Series:
        return self.df.isnull().sum()

    def check_duplicates(self) -> int:
        return self.df.duplicated().sum()

    def check_data_types(self) -> pd.Series:
        return self.df.dtypes

quality_checker = DataQualityChecks(df)

print("Missing values: \n", quality_checker.check_missing_values())
print("\nDuplicate rows: ", quality_checker.check_duplicates())
print("\nData types: \n", quality_checker.check_data_types())

## Correlation Analysis

In [ ]:
# Correlation Matrix
# Select only numeric columns to avoid string conversion errors
numeric_df = df.select_dtypes(include=[np.number])

plt.figure(figsize=(20, 16))
sns.heatmap(numeric_df.corr(), annot=False, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

numeric_df.corr()